<h1> <b> Purpose of this analysis</b></h1>
Purpose of this mini project is to experiment some of data wrangling techniques. We have rcieved Lending Club Loan Data set from 2007 to 2015. There are total 2.2 million rows and 145 columns. It is important to do some pre-processing work in order to analyze data and fill out missing values.  

<h2> <b> Packages</b></h2>
We will start by importing some of packages. Following packages will be imported.


In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

sns.set_style('whitegrid')

<h2> Step 1 : Reading Data in Dataframe </h2>
Now we will be reading data from "Loan.csv" to Pandas Dataframe.

In [18]:
data_list = []

for chunk in  pd.read_csv('loan.csv', chunksize=200000, low_memory=False):
    data_list.append(chunk)

data = pd.concat(data_list, axis= 0)
del data_list

# data = pd.read_csv("loan.csv", low_memory=False)



<h2> Step 2 : Checking data size </h2>
Let's quickly check the shape of data.This will give us idea as how large is current dataset. As we can see, current data set contains more than 2 million rows and 145 columns. Not all columns are useful for our analysis. 

In [19]:
# print(list(data.columns))
print(data.shape)

(2260668, 145)


<h2> Step 3 : Calculating Percentage of missing data per column </h2>
Here  we will be creating Dataframe called "df_null". This Data frame consist of column name with percent of missing data in each column in descending order. It is noted that any column that has more than 60% of data missing are useless for our analysis and we will simply drop it. 
As we see some column like "url", "id", "member_id" has 100% missing values. Apart from that lot of columns have more than 80% missing data. We will simply drop column which has more than 60% data missing. 

In [20]:
# print((data.isna().sum()[data.isna().sum() > 0]))
df_null = pd.DataFrame({'Count': data.isnull().sum(), 'Percent': 100*data.isnull().sum()/len(data)})
print(df_null[df_null['Percent'] > 0].sort_values(by='Percent', ascending=False))



# print(data.head(3))

                                              Count  Percent
id                                          2260668   100.00
url                                         2260668   100.00
member_id                                   2260668   100.00
orig_projected_additional_accrued_interest  2252242    99.63
hardship_length                             2250055    99.53
hardship_reason                             2250055    99.53
hardship_status                             2250055    99.53
deferral_term                               2250055    99.53
hardship_amount                             2250055    99.53
hardship_start_date                         2250055    99.53
hardship_end_date                           2250055    99.53
payment_plan_start_date                     2250055    99.53
hardship_loan_status                        2250055    99.53
hardship_dpd                                2250055    99.53
hardship_payoff_balance_amount              2250055    99.53
hardship_last_payment_am

<h2> Step 4 : Dropping column with more than 60% missing data </h2>
We will create a list of columns that a data missing more than 60%. We will simply drop those columns and create new dataframe called df_clean. We wil print shape of new dataframe to see how much data has been reduced. We can see total count of columns from 145 to reduced to 103. 

In [21]:
# Creating list of column with more than 60% data missing. 
list_60 = list(df_null[df_null['Percent']>60].index)

df_clean = data.drop(list_60, axis=1)
print(df_clean.shape)

(2260668, 103)


<h2> Step 5 : Analyzing columns with missing data  more than 10% and 60% </h2>
Now that we have eliminated major columns that had missing values. Lets analyze remaining column with missing values. For that we will print all column that has missing data and see how we can predict missing values. We will predict all columns that has missing data grether than 10%. Since we already dropped columns that has missing data more than 60%, in our dataframe.

In [22]:
df_null = pd.DataFrame({'Count': df_clean.isnull().sum(), 'Percent': 100*df_clean.isnull().sum()/len(df_clean)})
print(df_null[df_null['Percent'] > 10])


                          Count  Percent
mths_since_last_delinq  1158502    51.25
next_pymnt_d            1303607    57.66
open_acc_6m              866130    38.31
open_act_il              866129    38.31
open_il_12m              866129    38.31
open_il_24m              866129    38.31
mths_since_rcnt_il       909924    40.25
total_bal_il             866129    38.31
il_util                 1068850    47.28
open_rv_12m              866129    38.31
open_rv_24m              866129    38.31
max_bal_bc               866129    38.31
all_util                 866348    38.32
inq_fi                   866129    38.31
total_cu_tl              866130    38.31
inq_last_12m             866130    38.31
mths_since_recent_inq    295435    13.07


<h2> Step 5A : Checking data in columns with missing data more than 10% </h2>
Lets examine few values of this columns so we can get idea what kind of data estimation we need to do.

In [23]:
col_10 = list(df_null[df_null['Percent'] > 10].index)
print(df_clean[col_10].head(3))

   mths_since_last_delinq next_pymnt_d  open_acc_6m  open_act_il  open_il_12m  \
0                     nan     Mar-2019         2.00         2.00         1.00   
1                   71.00     Mar-2019         4.00         4.00         2.00   
2                     nan     Mar-2019         0.00         1.00         0.00   

   open_il_24m  mths_since_rcnt_il  total_bal_il  il_util  open_rv_12m  \
0         2.00                2.00      12560.00    69.00         2.00   
1         3.00                3.00      87153.00    88.00         4.00   
2         2.00               14.00       7150.00    72.00         0.00   

   open_rv_24m  max_bal_bc  all_util  inq_fi  total_cu_tl  inq_last_12m  \
0         7.00     2137.00     28.00    1.00        11.00          2.00   
1         5.00      998.00     57.00    2.00        15.00          2.00   
2         2.00        0.00     35.00    1.00         5.00          0.00   

   mths_since_recent_inq  
0                   2.00  
1                   4.0

<h2> Step 5B : Dropping payment dates columns </h2>
As we see above data set most missing data is numeric and we can use mode or median in order to fill missing values. By looking at dataset and name of columns, mode will be better option compared to median for most of columns. For "total_bil_il" we will use median. Lets analyze mode and median of those columns. But before we need to drop 3 columns which is not useful for our analysis. Those are 
1)  "next_pymnt_dt
2) 'last_credit_pull_d'
3) 'last_pymnt_d'

Lets create a list of these columns. 

In [24]:
# print(df_clean.shape)
cols_to_drop = ['next_pymnt_d', 'last_credit_pull_d', 'last_pymnt_d']
df_clean.drop(cols_to_drop, axis=1, inplace=True)
df_null = pd.DataFrame({'Count': df_clean.isnull().sum(), 'Percent': 100*df_clean.isnull().sum()/len(df_clean)})

<h2> Step 5C : Checking mode and median of missing data columns </h2>
Now we will eamine mode and median of all columns those we trying to do estiamtion.



In [25]:
col_10 = list(df_null[df_null['Percent'] > 10].index)
# print(col_10)
# print(df_null.loc[col_10, :])
for index in col_10:
    df_null.loc[index, 'mode'] = df_clean[index].mode()[0]
    df_null.loc[index, 'median'] = df_clean[index].median()
print(df_null.loc[col_10, :])



                          Count  Percent  mode   median
mths_since_last_delinq  1158502    51.25 12.00    31.00
open_acc_6m              866130    38.31  0.00     1.00
open_act_il              866129    38.31  1.00     2.00
open_il_12m              866129    38.31  0.00     0.00
open_il_24m              866129    38.31  1.00     1.00
mths_since_rcnt_il       909924    40.25  7.00    13.00
total_bal_il             866129    38.31  0.00 23127.00
il_util                 1068850    47.28 78.00    72.00
open_rv_12m              866129    38.31  0.00     1.00
open_rv_24m              866129    38.31  1.00     2.00
max_bal_bc               866129    38.31  0.00  4413.00
all_util                 866348    38.32 59.00    58.00
inq_fi                   866129    38.31  0.00     1.00
total_cu_tl              866130    38.31  0.00     0.00
inq_last_12m             866130    38.31  0.00     1.00
mths_since_recent_inq    295435    13.07  1.00     5.00


<h2> Step 5D : predicting missing values using mode </h2>
By looking at above table, it makes more sense to fill all above columns by mode rather than median. Lets fill those missing values in above columns using mode.

In [26]:
df_clean[col_10] = df_clean[col_10].fillna(df_clean.mode().iloc[0])

<h2> Step 6 : Dropping all missing data </h2>
Now lets examine remaining missing data. Most columns will have missing data less than 10%. We will simply drop them. We may see lot of columns having missing data less than 10%. We will simply drop them. That way we will only lose 10% of data

In [27]:
# df_null = pd.DataFrame({'Count': df_clean.isnull().sum(), 'Percent': 100*df_clean.isnull().sum()/len(df_clean)})
# print(df_null[df_null['Percent'] > 0])
df_clean.dropna(inplace=True)
print(df_clean.shape)


(1854073, 100)


<h2> Step 7 : Analyzing datatypes of clean data </h2>
Now we have get rid of all missing data. Dataset is almost clean. Lets see datatypes of datasets. 

In [28]:
print(df_clean.dtypes)

loan_amnt                       int64
funded_amnt                     int64
funded_amnt_inv               float64
term                           object
int_rate                      float64
installment                   float64
grade                          object
sub_grade                      object
emp_title                      object
emp_length                     object
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_d                        object
loan_status                    object
pymnt_plan                     object
purpose                        object
title                          object
zip_code                       object
addr_state                     object
dti                           float64
delinq_2yrs                   float64
earliest_cr_line               object
inq_last_6mths                float64
mths_since_last_delinq        float64
open_acc                      float64
pub_rec     

<h2> Step 8 : Fixing "Employment Length" column </h2>
In above datatypes one of the important title is employment length. We should be expecting it as integer or float but it is object type. Lets analyze this column.

In [29]:
print(df_clean['emp_length'].value_counts())

10+ years    665309
2 years      179041
3 years      158836
< 1 year     157037
1 year       129053
5 years      121646
4 years      119208
6 years       89893
7 years       81825
8 years       81520
9 years       70705
Name: emp_length, dtype: int64


Now lets remove unnecessary keywords suh as "year", "<" and "+". We will consider all records of empoyment length <1 years as 0 years and 10+ years as 10.

In [30]:
# df_clean['emp_length'].astype('str').dtypes
df_clean['emp_length'] = df_clean['emp_length'].str.replace(r'[+]\s|[a-z]', '')
df_clean['emp_length'] = df_clean['emp_length'].str.replace('< 1', '0')
# df_clean['emp_length'] = df_clean['emp_length'].str.replace('10+ ', '10')

print(df_clean['emp_length'].value_counts())



10    665309
2     179041
3     158836
0     157037
1     129053
5     121646
4     119208
6      89893
7      81825
8      81520
9      70705
Name: emp_length, dtype: int64


<h2> Step 9 : Analyzing clean data </h2>
Now that we have clean data, we will analyze its shape. Majority of columns are numeric and hence we will use dataframe.describe function to just see summary of numeric columns. 

In [31]:

print(df_clean.shape)
pd.options.display.float_format = '{:.2f}'.format
print(df_clean.describe())

(1854073, 100)
       loan_amnt  funded_amnt  funded_amnt_inv   int_rate  installment  \
count 1854073.00   1854073.00       1854073.00 1854073.00   1854073.00   
mean    15425.32     15425.30         15420.12      13.04       455.20   
std      9248.70      9248.70          9246.58       4.85       268.33   
min      1000.00      1000.00           725.00       5.31         4.93   
25%      8000.00      8000.00          8000.00       9.44       260.07   
50%     13875.00     13875.00         13825.00      12.62       388.14   
75%     20000.00     20000.00         20000.00      15.88       605.58   
max     40000.00     40000.00         40000.00      30.99      1719.83   

        annual_inc        dti  delinq_2yrs  inq_last_6mths  \
count   1854073.00 1854073.00   1854073.00      1854073.00   
mean      80628.03      18.95         0.31            0.54   
std      111555.69      11.56         0.88            0.84   
min           0.00      -1.00         0.00            0.00   
25%     

<h2> Step 10: Exporting clean data to 'csv"

In [32]:
df_clean.to_csv("Loan_cleandata.csv")